In [1]:
%cd ../model/

/home/chisphung/ALPR_Fisheye/model


In [8]:
import os
import numpy as np
from collections import defaultdict
from yolo.ultralytics.utils.nms import non_max_suppression
# from ai_core.object_detection.yolov5_custom.od.data.datasets import letterbox
from typing import List
# from dynaconf import settings
from yolo.ultralytics.models.yolo.model import YOLO
import cv2

In [17]:
model = YOLO('../weight/recognition_v11m.pt')

In [18]:
res = model.predict('../dataset/test/images/')



image 1/825 /home/chisphung/ALPR_Fisheye/model/../dataset/test/images/CarLongPlate119_jpg.rf.75074a43a0dbf87e62aeac19c12bce52.jpg: 512x768 1 BSD, 31.8ms
image 1/825 /home/chisphung/ALPR_Fisheye/model/../dataset/test/images/CarLongPlate119_jpg.rf.75074a43a0dbf87e62aeac19c12bce52.jpg: 512x768 1 BSD, 31.8ms
image 2/825 /home/chisphung/ALPR_Fisheye/model/../dataset/test/images/CarLongPlate136_jpg.rf.b0bebaf7ef889683ec2c7b3459c707e2.jpg: 512x768 1 BSD, 31.1ms
image 2/825 /home/chisphung/ALPR_Fisheye/model/../dataset/test/images/CarLongPlate136_jpg.rf.b0bebaf7ef889683ec2c7b3459c707e2.jpg: 512x768 1 BSD, 31.1ms
image 3/825 /home/chisphung/ALPR_Fisheye/model/../dataset/test/images/CarLongPlate13_jpg.rf.b4a36ada848f4f6c6678ea002d4b3264.jpg: 512x768 1 BSD, 31.1ms
image 3/825 /home/chisphung/ALPR_Fisheye/model/../dataset/test/images/CarLongPlate13_jpg.rf.b4a36ada848f4f6c6678ea002d4b3264.jpg: 512x768 1 BSD, 31.1ms
image 4/825 /home/chisphung/ALPR_Fisheye/model/../dataset/test/images/CarLongPlate

In [19]:

def yolo_txt_to_xyxy(lines, img_w, img_h):
    boxes = []
    classes = []
    for line in lines:
        if not line.strip():
            continue
        cls, xc, yc, w, h = map(float, line.strip().split())
        x1 = (xc - w/2) * img_w
        y1 = (yc - h/2) * img_h
        x2 = (xc + w/2) * img_w
        y2 = (yc + h/2) * img_h
        boxes.append([x1, y1, x2, y2])
        classes.append(int(cls))
    return np.array(boxes, dtype=np.float32), np.array(classes, dtype=int)

def box_iou_matrix(a, b):
    # a: [N,4], b:[M,4]
    if len(a)==0 or len(b)==0:
        return np.zeros((len(a), len(b)), dtype=np.float32)
    inter_x1 = np.maximum(a[:,None,0], b[None,:,0])
    inter_y1 = np.maximum(a[:,None,1], b[None,:,1])
    inter_x2 = np.minimum(a[:,None,2], b[None,:,2])
    inter_y2 = np.minimum(a[:,None,3], b[None,:,3])
    inter_w = np.clip(inter_x2 - inter_x1, 0, None)
    inter_h = np.clip(inter_y2 - inter_y1, 0, None)
    inter = inter_w * inter_h
    area_a = (a[:,2]-a[:,0]) * (a[:,3]-a[:,1])
    area_b = (b[:,2]-b[:,0]) * (b[:,3]-b[:,1])
    union = area_a[:,None] + area_b[None,:] - inter
    return inter / np.clip(union, 1e-9, None)

def compute_ap(rec, prec):
    # 101-point interpolation
    mrec = np.concatenate(([0.0], rec, [1.0]))
    mpre = np.concatenate(([0.0], prec, [0.0]))
    for i in range(mpre.size-1, 0, -1):
        mpre[i-1] = np.maximum(mpre[i-1], mpre[i])
    x = np.linspace(0,1,101)
    ap = np.trapz(np.interp(x, mrec, mpre), x)
    return ap

def evaluate_results(results, labels_dir, iou_thres=0.5, merge_plate_classes=True, dedup_iou=0.6):
    """
    merge_plate_classes: if True, map class 1 -> 0 (treat long/short plate as one class)
    dedup_iou: if >0, after merging remove duplicate predictions of same class in one image
    """
    per_class_dets = defaultdict(list)  # class -> list of (conf, TP)
    gt_counter = defaultdict(int)

    for r in results:
        img_path = r.path
        img_h, img_w = r.orig_shape
        base = os.path.splitext(os.path.basename(img_path))[0]
        gt_file = os.path.join(labels_dir, base + ".txt")
        if os.path.exists(gt_file):
            with open(gt_file, 'r') as f:
                gt_boxes, gt_cls = yolo_txt_to_xyxy(f.readlines(), img_w, img_h)
        else:
            gt_boxes = np.zeros((0,4), dtype=np.float32)
            gt_cls = np.array([], dtype=int)

        if merge_plate_classes:
            gt_cls = np.where(gt_cls == 1, 0, gt_cls)

        for c in gt_cls:
            gt_counter[c] += 1

        if r.boxes is None or len(r.boxes) == 0:
            continue

        pred_boxes = r.boxes.xyxy.cpu().numpy()
        pred_confs = r.boxes.conf.cpu().numpy()
        pred_cls = r.boxes.cls.cpu().numpy().astype(int)

        if merge_plate_classes:
            pred_cls = np.where(pred_cls == 1, 0, pred_cls)

        # Optional dedup after merge (keep highest conf for overlapping same-class boxes)
        if dedup_iou and dedup_iou > 0:
            keep = []
            order = np.argsort(-pred_confs)
            for idx in order:
                b = pred_boxes[idx]
                c = pred_cls[idx]
                duplicate = False
                for k in keep:
                    if pred_cls[k] == c:
                        # IoU with kept box
                        kb = pred_boxes[k]
                        inter_x1 = max(b[0], kb[0])
                        inter_y1 = max(b[1], kb[1])
                        inter_x2 = min(b[2], kb[2])
                        inter_y2 = min(b[3], kb[3])
                        iw = max(0, inter_x2 - inter_x1)
                        ih = max(0, inter_y2 - inter_y1)
                        inter = iw * ih
                        ub = (b[2]-b[0])*(b[3]-b[1]) + (kb[2]-kb[0])*(kb[3]-kb[1]) - inter
                        iou = inter / (ub + 1e-9)
                        if iou >= dedup_iou:
                            duplicate = True
                            break
                if not duplicate:
                    keep.append(idx)
            pred_boxes = pred_boxes[keep]
            pred_confs = pred_confs[keep]
            pred_cls = pred_cls[keep]

        used_gt = set()
        for pb, pc, conf in sorted(zip(pred_boxes, pred_cls, pred_confs), key=lambda x: -x[2]):
            cls_mask = (gt_cls == pc)
            gt_boxes_cls = gt_boxes[cls_mask]
            gt_indices = np.where(cls_mask)[0]
            tp = 0
            if len(gt_boxes_cls):
                ious = box_iou_matrix(np.array([pb]), gt_boxes_cls)[0]
                best_i = np.argmax(ious)
                if ious[best_i] >= iou_thres:
                    global_gt_index = gt_indices[best_i]
                    if (pc, global_gt_index) not in used_gt:
                        tp = 1
                        used_gt.add((pc, global_gt_index))
            per_class_dets[pc].append((conf, tp))

    summary = []
    map_sum = 0.0
    class_count = 0
    for cls, dets in per_class_dets.items():
        dets.sort(key=lambda x: -x[0])
        tps = np.array([d[1] for d in dets])
        fps = 1 - tps
        tp_cum = np.cumsum(tps)
        fp_cum = np.cumsum(fps)
        total_gt = gt_counter.get(cls, 0)
        if total_gt == 0:
            continue
        recalls = tp_cum / total_gt
        precisions = tp_cum / np.clip(tp_cum + fp_cum, 1e-9, None)
        ap = compute_ap(recalls, precisions)
        map_sum += ap
        class_count += 1
        summary.append({
            "class": cls,
            "gt": total_gt,
            "detections": len(dets),
            f"AP@{iou_thres:.2f}": ap,
            "precision_last": precisions[-1],
            "recall_last": recalls[-1]
        })

    mAP = map_sum / class_count if class_count else 0.0
    return summary, mAP

# Re-run evaluation with merged classes
labels_dir = "../dataset/test/labels"
summary, mAP = evaluate_results(res, labels_dir, iou_thres=0.5, merge_plate_classes=True)

print("Per-class summary (after merging class 1 -> 0):")
for row in summary:
    print(row)
print(f"\nOverall mAP@0.50 (merged): {mAP:.4f}")


Per-class summary (after merging class 1 -> 0):
{'class': np.int64(0), 'gt': 1140, 'detections': 788, 'AP@0.50': np.float64(0.8296706293419172), 'precision_last': np.float64(0.9720812182741116), 'recall_last': np.float64(0.6719298245614035)}

Overall mAP@0.50 (merged): 0.8297


In [13]:
model.export(format="TensorRT", half=True)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
WARNING ⚠️ WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.202 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3904MiB)
Ultralytics 8.3.202 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3904MiB)

PyTorch: starting from '../weight/recognition_v11m.pt' with input shape (1, 3, 768, 768) BCHW and output shape(s) (1, 6, 12096) (115.5 MB)

PyTorch: starting from '../weight/recognition_v11m.pt' with input shape (1, 3, 768, 768) BCHW and output shape(s) (1, 6, 12096) (115.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.67', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.67', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/18.2 MB ? eta -:--:--
   ━╸━━━━━━━━━━

'../weight/recognition_v11m.engine'

In [20]:
model = YOLO('../weight/recognition_v11m.engine')

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
WARNING ⚠️ WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


## Inference with tensorRT model

- Achieved approx 107 FPS on RTX 3060 laptop GPU

In [21]:
res = model.predict('../dataset/test/images/')

Loading ../weight/recognition_v11m.engine for TensorRT inference...
Loading ../weight/recognition_v11m.engine for TensorRT inference...
[09/30/2025-00:11:01] [TRT] [W] WARNING The logger passed into createInferRuntime differs from one already registered for an existing builder, runtime, or refitter. So the current new logger is ignored, and TensorRT will use the existing one which is returned by nvinfer1::getLogger() instead.
[09/30/2025-00:11:01] [TRT] [I] Loaded engine size: 43 MiB
[09/30/2025-00:11:01] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +48, now: CPU 1, GPU 171 (MiB)


image 1/825 /home/chisphung/ALPR_Fisheye/model/../dataset/test/images/CarLongPlate119_jpg.rf.75074a43a0dbf87e62aeac19c12bce52.jpg: 768x768 1 BSD, 12.8ms
image 1/825 /home/chisphung/ALPR_Fisheye/model/../dataset/test/images/CarLongPlate119_jpg.rf.75074a43a0dbf87e62aeac19c12bce52.jpg: 768x768 1 BSD, 12.8ms
image 2/825 /home/chisphung/ALPR_Fisheye/model/../da

In [22]:
summary, mAP = evaluate_results(res, labels_dir, iou_thres=0.5, merge_plate_classes=True)

print("Per-class summary (after merging class 1 -> 0):")
for row in summary:
    print(row)
print(f"\nOverall mAP@0.50 (merged): {mAP:.4f}")


Per-class summary (after merging class 1 -> 0):
{'class': np.int64(0), 'gt': 1140, 'detections': 794, 'AP@0.50': np.float64(0.8291747332195383), 'precision_last': np.float64(0.9659949622166247), 'recall_last': np.float64(0.6728070175438596)}

Overall mAP@0.50 (merged): 0.8292
